# Lab | Customer Analysis Round 6

For this lab, we still keep using the `marketing_customer_analysis.csv` file that you can find in the `files_for_lab` folder.

### Get the data

We are using the `marketing_customer_analysis.csv` file.

### Dealing with the data

Already done in the round 2.

### Explore the data

Done in the round 3.

### Processing Data

(_Further processing..._)

- X-y split. (_done_)
- Normalize (numerical). (_done_)
- One Hot/Label Encoding (categorical).
- Concat DataFrames

### Linear Regression

- Train-test split.
- Apply linear regression.

### Model Validation

- Description:
  - R2.
  - MSE.
  - RMSE.
  - MAE.


In [225]:
# Importing the libraries

import pandas as pd

import numpy as np

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [245]:
# Importing the dataframe

customer_df = pd.read_csv("/Users/leozinho.air/Desktop/ironhack_da/class_04/lab-customer-analysis-round-2/files_for_lab/csv_files/marketing_customer_analysis.csv")

# Cleaning operations

columns = []

for i in range(len(customer_df.columns)):
    columns.append(customer_df.columns[i].lower().replace(' ', '_'))

customer_df.columns = columns

customer_df

# Dropping the column 'unnamed:_0'

customer_df = customer_df.drop(['unnamed:_0'], axis = 1)

# The only values are Nan or vehicle class 'A', let's create a new df without the column.

customer_df = customer_df.drop(['vehicle_type'], axis = 1)

customer_df

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,48029,...,52,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize
1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,0,...,26,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,22139,...,31,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize
3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,49078,...,3,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize
4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,23675,...,31,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,0,...,40,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize
10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,61146,...,68,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize
10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,39837,...,63,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize
10908,WA60547,California,11971.977650,No,Premium,College,2/13/11,Employed,F,64195,...,27,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize


In [246]:
# This is for predictions
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# These Libs are for stats -> this ones are for description
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Creating a dataframe with only numerical values

num_cust = customer_df.select_dtypes(include = np.number)

# Checking for null values

num_cust.isnull().any() # Two columns have nan values

# I chose to fill the nan valus with the means of the two columns

mean_months = num_cust['months_since_last_claim'].mean()

num_cust['months_since_last_claim'].fillna(value = mean_months, inplace = True)


mean_complaints = num_cust['number_of_open_complaints'].mean()
num_cust['number_of_open_complaints'].fillna(value = mean_complaints, inplace = True)

num_cust.isnull().sum().sum() # There are no nan values




0

In [247]:
# X-y split.

Y = customer_df['total_claim_amount']
X = customer_df.drop(['total_claim_amount'], axis = 1)

# Normalizing the DataFrame

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() # This is the normalization process

normalized_data = scaler.fit_transform(num_cust) # Fit it to the data
normalized_data = pd.DataFrame(normalized_data, columns = num_cust.columns)

normalized_data

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0.035752,0.480381,0.000000,0.200000,0.525253,0.000000,1.000,0.101171
1,0.004059,0.000000,0.012658,0.085714,0.262626,0.000000,0.000,0.257445
2,0.160264,0.221432,0.164557,0.971429,0.313131,0.000000,0.125,0.165875
3,0.250953,0.490873,0.151899,0.285714,0.030303,0.000000,0.125,0.167263
4,0.087527,0.236795,0.236287,0.432831,0.313131,0.076851,0.750,0.244657
...,...,...,...,...,...,...,...,...
10905,0.167823,0.000000,0.810127,0.432831,0.404040,0.076851,0.750,0.419717
10906,0.041281,0.611576,0.016878,0.200000,0.686869,0.000000,0.625,0.094333
10907,0.270122,0.398446,0.590717,0.314286,0.636364,0.000000,0.125,0.131763
10908,0.123717,0.642072,0.409283,0.000000,0.272727,0.800000,0.625,0.213674


In [248]:
# One Hot/Label Encoding (categorical)

# Creating a dataframe with categorial values

cat_cust = customer_df.select_dtypes(include = 'object')

# Printing unique values of the df for better understanding

columns = ['customer', 'state', 'response', 'coverage', 'education',
       'effective_to_date', 'employmentstatus', 'gender', 'location_code',
       'marital_status', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'vehicle_class', 'vehicle_size']

for column in columns:
    is_unique = cat_cust[column].unique()
    print(f'The unique values in {column} are : ', is_unique )
    
'''
The unique values in customer are :  ['DK49336' 'KX64629' 'LZ68649' ... 'KX53892' 'TL39050' 'WA60547']
The unique values in state are :  ['Arizona' 'California' 'Washington' 'Oregon' nan 'Nevada']
The unique values in response are :  ['No' 'Yes' nan]
The unique values in coverage are :  ['Basic' 'Extended' 'Premium']
The unique values in education are :  ['College' 'Bachelor' 'High School or Below' 'Doctor' 'Master']
The unique values in effective_to_date are :  ['2/18/11' '1/18/11' '2/10/11' '1/11/11' '1/17/11' '2/14/11' '2/24/11'
 '1/19/11' '1/4/11' '1/2/11' '2/7/11' '1/31/11' '1/26/11' '2/28/11'
 '1/16/11' '2/26/11' '2/23/11' '1/15/11' '2/2/11' '2/15/11' '1/24/11'
 '2/21/11' '2/22/11' '1/7/11' '1/28/11' '2/8/11' '2/12/11' '2/20/11'
 '1/5/11' '2/19/11' '1/3/11' '2/3/11' '1/22/11' '1/23/11' '2/5/11'
 '2/13/11' '1/25/11' '2/16/11' '2/1/11' '1/27/11' '1/12/11' '1/20/11'
 '2/6/11' '2/11/11' '1/21/11' '1/29/11' '1/9/11' '2/9/11' '2/27/11'
 '1/1/11' '2/17/11' '2/25/11' '1/13/11' '1/6/11' '2/4/11' '1/14/11'
 '1/10/11' '1/8/11' '1/30/11']
The unique values in employmentstatus are :  ['Employed' 'Unemployed' 'Medical Leave' 'Disabled' 'Retired']
The unique values in gender are :  ['M' 'F']
The unique values in location_code are :  ['Suburban' 'Urban' 'Rural']
The unique values in marital_status are :  ['Married' 'Single' 'Divorced']
The unique values in policy_type are :  ['Corporate Auto' 'Personal Auto' 'Special Auto']
The unique values in policy are :  ['Corporate L3' 'Personal L3' 'Personal L2' 'Corporate L2' 'Personal L1'
 'Special L1' 'Corporate L1' 'Special L3' 'Special L2']
The unique values in renew_offer_type are :  ['Offer3' 'Offer4' 'Offer2' 'Offer1']
The unique values in sales_channel are :  ['Agent' 'Call Center' 'Branch' 'Web']
The unique values in vehicle_class are :  ['Four-Door Car' 'SUV' 'Two-Door Car' 'Sports Car' 'Luxury Car'
 'Luxury SUV' nan]
The unique values in vehicle_size are :  ['Medsize' 'Small' 'Large' nan]
'''

# Dropping the categorical columns that i will not use on the model

cat_cust = cat_cust.drop(['customer','effective_to_date','gender','renew_offer_type', 'vehicle_size'], axis=1)
cat_cust

# Dealing with nan values

cat_cust.isnull().sum() # There are more or less 600 missing values for each of these columns: state, response, vehicle class

# I will drop the nan values
cat_cust = cat_cust.dropna()

cat_cust.isnull().sum() # There are no nan now


The unique values in customer are :  ['DK49336' 'KX64629' 'LZ68649' ... 'KX53892' 'TL39050' 'WA60547']
The unique values in state are :  ['Arizona' 'California' 'Washington' 'Oregon' nan 'Nevada']
The unique values in response are :  ['No' 'Yes' nan]
The unique values in coverage are :  ['Basic' 'Extended' 'Premium']
The unique values in education are :  ['College' 'Bachelor' 'High School or Below' 'Doctor' 'Master']
The unique values in effective_to_date are :  ['2/18/11' '1/18/11' '2/10/11' '1/11/11' '1/17/11' '2/14/11' '2/24/11'
 '1/19/11' '1/4/11' '1/2/11' '2/7/11' '1/31/11' '1/26/11' '2/28/11'
 '1/16/11' '2/26/11' '2/23/11' '1/15/11' '2/2/11' '2/15/11' '1/24/11'
 '2/21/11' '2/22/11' '1/7/11' '1/28/11' '2/8/11' '2/12/11' '2/20/11'
 '1/5/11' '2/19/11' '1/3/11' '2/3/11' '1/22/11' '1/23/11' '2/5/11'
 '2/13/11' '1/25/11' '2/16/11' '2/1/11' '1/27/11' '1/12/11' '1/20/11'
 '2/6/11' '2/11/11' '1/21/11' '1/29/11' '1/9/11' '2/9/11' '2/27/11'
 '1/1/11' '2/17/11' '2/25/11' '1/13/11' '1/6/11' '

state               0
response            0
coverage            0
education           0
employmentstatus    0
location_code       0
marital_status      0
policy_type         0
policy              0
sales_channel       0
vehicle_class       0
dtype: int64

In [249]:
# One Hot Encoding -> 'state', 'response', 'coverage', 'education', 'employmentstatus','location_code', 'marital_status', 'policy_type', 'policy','sales_channel', 'vehicle_class'

# One hot encoding for the target columns

cat_cust = cat_cust[['state', 'response', 'coverage', 'education', 'employmentstatus',
       'location_code', 'marital_status', 'policy_type', 'policy',
       'sales_channel', 'vehicle_class']]

#categorical_cols = ['state', 'response', 'coverage', 'education', 'employmentstatus',
       #'location_code', 'marital_status', 'policy_type', 'policy',
      # 'sales_channel', 'vehicle_class']

cat_encoded = pd.get_dummies(cat_cust, drop_first= False)

cat_encoded # Succesfully encoded

cat_encoded = cat_encoded.reset_index(drop = True)

cat_encoded.isnull().sum().sum() # no nans after encoding

0

In [250]:

# Concate the two DataFrames

cust_encoded = pd.concat([num_cust,cat_encoded], axis = 1)

cust_encoded.isnull().sum()

# I got some nan values, i do not know why it happens.

cust_encoded = cust_encoded.dropna()

In [251]:
# Linear Regression

# Train-test split.

from sklearn.model_selection import train_test_split

features = [col for col in cust_encoded.columns]


X = cust_encoded.loc[:, features]
y = cust_encoded.loc[:, ['total_claim_amount']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .80) # this code create 2 df with indipendet vars and 2 set dependent var





In [235]:
# Apply linear regression -> with the concatenated df (numerical + categorical chosen)
from sklearn import linear_model

Y = cust_encoded['total_claim_amount']
X = cust_encoded.drop(['total_claim_amount'], axis = 1)

# The linear model has the formula Y = a * X + b, with 'add_constant' we force the model to give a costant

X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     257.6
Date:                Mon, 16 Oct 2023   Prob (F-statistic):               0.00
Time:                        20:40:25   Log-Likelihood:                -65103.
No. Observations:                9688   AIC:                         1.303e+05
Df Residuals:                    9645   BIC:                         1.306e+05
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             15.8603      2.837      5.590      0.000      10.299      21.422
customer_lifetime_value           -0.0005      0.000     -1.522      0.128      -0.001       0.000
income                            -0.0034   6.75e-05    -49.829      0.000      -0.003      -0.003
monthly_premium_auto               5.3919      0.065     83.067      0.000       5.265       5.519
months_since_last_claim            0.0094      0.210      0.045      0.964      -0.402       0.420
months_since_policy_inception     -0.1779      0.073     -2.422      0.015      -0.322      -0.034
number_of_open_complaints          0.4047      2.343      0.173      0.863      -4.188       4.998
number_of_policies                 0.0777      0.854      0.091      0.928      -1.597       1.752
state_Arizona                      2.2373      4.384      0.510      0.610      -6.357      10.832
state_California                   3.1530      3.598      0.876      0.381      -3.899      10.205
state_Nevada                       6.7522      5.684      1.188      0.235      -4.390      17.894
state_Oregon                      -2.6737      3.796     -0.704      0.481     -10.115       4.767
state_Washington                   6.3915      5.927      1.078      0.281      -5.227      18.010
response_No                        3.1759      3.236      0.981      0.326      -3.168       9.519
response_Yes                      12.6844      3.604      3.519      0.000       5.619      19.750
coverage_Basic                     8.9651      3.130      2.864      0.004       2.829      15.101
coverage_Extended                  0.3514      3.539      0.099      0.921      -6.585       7.288
coverage_Premium                   6.5438      5.022      1.303      0.193      -3.300      16.388
education_Bachelor                 5.6680      4.072      1.392      0.164      -2.313      13.649
education_College                  3.7069      4.100      0.904      0.366      -4.331      11.745
education_Doctor                   3.8345      8.901      0.431      0.667     -13.614      21.283
education_High School or Below     3.4351      4.192      0.819      0.413      -4.783      11.653
education_Master                  -0.7842      6.331     -0.124      0.901     -13.193      11.625
employmentstatus_Disabled         -5.0274      8.485     -0.592      0.554     -21.661      11.606
employmentstatus_Employed          9.4793      4.428      2.141      0.032       0.799      18.160
employmentstatus_Medical Leave     4.2715      8.245      0.518      0.604     -11.891      20.434
employmentstatus_Retired          -4.9602     10.360     -0.479      0.632     -25.269      15.349
employmentstatus_Unemployed       12.0971      5.216      2.319      0.020       1.872      22.322
location_code_

In [236]:
'''
Dependent Variable: This section identifies the dependent variable (the one you're trying to predict) and the model used 
(OLS - Ordinary Least Squares).

R-squared: This is a measure of how well the independent variables explain the variation in the dependent variable. 
It ranges from 0 to 1, with higher values indicating a better fit. An R-squared value of 1 means that the model perfectly 
fits the data.

Adj. R-squared: This is a version of R-squared that adjusts for the number of predictors in the model. 
It can be more informative when comparing models with different numbers of predictors.


coef: This column lists the coefficients of the independent variables. These coefficients represent the estimated effect 
of each independent variable on the dependent variable.

std err: Standard error of the coefficient estimates, which measures the variability of the coefficient.

t: The t-statistic measures how many standard errors the coefficient is away from zero. 
Larger absolute values indicate greater significance.

P>|t|: This is the p-value associated with the t-statistic. It measures the significance of each coefficient. 
Lower values (typically below 0.05) indicate greater significance.

[0.025, 0.975]: These columns give the 95% confidence interval for the coefficients. 
It helps you understand the range of likely values for the coefficient.

Intercept: If you included a constant term (intercept) in your model, you'll see its coefficient, 
standard error, t-statistic, p-value, and confidence interval.

Omnibus and Jarque-Bera: These tests are for the normality of the residuals. 
A low p-value in the Jarque-Bera test suggests that the residuals are not normally distributed.

Durbin-Watson: This test checks for the presence of autocorrelation in the residuals. 
A value close to 2 suggests no autocorrelation.

Cond. No.: This is the condition number, which measures multicollinearity. 
High values (above 30) indicate potential multicollinearity issues.

AIC and BIC: These are information criteria that help you compare the goodness of fit between different models. 
Lower values indicate better model fit.

F-statistic: This statistic tests the overall significance of the model. 
A low p-value indicates that at least one of the independent variables is significantly related to the dependent variable.

Prob (F-statistic): The p-value associated with the F-statistic. A low p-value suggests the model as a whole is significant.

Log-Likelihood: The log-likelihood of the model, which is used in model comparisons.
'''

"\nDependent Variable: This section identifies the dependent variable (the one you're trying to predict) and the model used \n(OLS - Ordinary Least Squares).\n\nR-squared: This is a measure of how well the independent variables explain the variation in the dependent variable. \nIt ranges from 0 to 1, with higher values indicating a better fit. An R-squared value of 1 means that the model perfectly \nfits the data.\n\nAdj. R-squared: This is a version of R-squared that adjusts for the number of predictors in the model. \nIt can be more informative when comparing models with different numbers of predictors.\n\n\ncoef: This column lists the coefficients of the independent variables. These coefficients represent the estimated effect \nof each independent variable on the dependent variable.\n\nstd err: Standard error of the coefficient estimates, which measures the variability of the coefficient.\n\nt: The t-statistic measures how many standard errors the coefficient is away from zero. \nLa

In [244]:
# Model Validation

# Description:R2 - MSE - RMSE - MAE

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

lm = linear_model.LinearRegression()
model = lm.fit(X,Y)
lm.score(X,Y)

predictions = lm.predict(X)

print("R2 value is = ",round(r2_score(Y, predictions),2))
print("The intercept of the model is = ",lm.intercept_)
print("The coefficients of the model are = ",lm.coef_)

mse = mean_squared_error(Y, predictions)
mae = mean_absolute_error(Y, predictions)
print("The mse of the model is = ",mse)
print("The root mse of the model is = ",np.sqrt(mse))
print("The mean absolute error of the model is = ",mae)


R2 value is =  0.53
The intercept of the model is =  62.8245573174562
The coefficients of the model are =  [ 4.35933797e-12 -4.89566242e-04 -3.36382030e-03  5.39188192e+00
  9.35024521e-03 -1.77930936e-01  4.04661851e-01  7.76702169e-02
 -9.34739548e-01 -1.90578487e-02  3.58013723e+00 -5.84574519e+00
  3.21940536e+00 -4.75424017e+00  4.75424017e+00  3.67836794e+00
 -4.93534240e+00  1.25697446e+00  2.49596553e+00  5.34880345e-01
  6.62464003e-01  2.62999239e-01 -3.95630911e+00 -8.19944018e+00
  6.30719132e+00  1.09945918e+00 -8.13221781e+00  8.92500749e+00
 -3.60849029e+00  3.49537543e+00  1.13114860e-01 -3.10364278e+00
  3.04190978e+00  6.17330018e-02 -5.16683220e-01 -1.83679722e+00
  2.35348044e+00 -1.96360506e+01  1.09943841e+01  8.12498327e+00
 -2.31471605e+00 -2.93236866e+00  3.41028748e+00 -1.83834770e+01
  4.94335469e+00  1.57936027e+01 -4.26007576e+00 -2.11375644e-01
  6.53431581e+00 -2.06286440e+00  5.22347492e+00 -2.02182526e+01
  1.48393264e+01  1.21792537e+00 -5.47788101e+00